In [3]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("C:/Users/cestm/Downloads/statistics/ranking.csv")

# Extract specific columns 'TEAM_ID', 'STANDINGSDATE', 'TEAM' and filter for 'CONFERENCE'=='West'
selected_columns = df[['TEAM_ID', 'STANDINGSDATE', 'TEAM', 'CONFERENCE']][df['CONFERENCE'] == 'West']

# Filter rows where 'STANDINGSDATE' ends with "-12-22"
filtered_rows = selected_columns[selected_columns['STANDINGSDATE'].str.endswith('-04-02')]

# Display the filtered DataFrame
print(filtered_rows)




           TEAM_ID STANDINGSDATE         TEAM CONFERENCE
0       1610612743    2022-12-22       Denver       West
1       1610612763    2022-12-22      Memphis       West
2       1610612740    2022-12-22  New Orleans       West
3       1610612756    2022-12-22      Phoenix       West
4       1610612746    2022-12-22  LA Clippers       West
...            ...           ...          ...        ...
120052  1610612743    2014-12-22       Denver       West
120053  1610612758    2014-12-22   Sacramento       West
120054  1610612762    2014-12-22         Utah       West
120055  1610612747    2014-12-22  L.A. Lakers       West
120056  1610612750    2014-12-22    Minnesota       West

[299 rows x 4 columns]


In [24]:
rankingyear2022 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2022-')]
rankingyear2021 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2021-')]
rankingyear2020 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2020-')]
rankingyear2019 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2019-')]
rankingyear2018 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2018-')]
rankingyear2017 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2017-')]
rankingyear2016 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2016-')]
rankingyear2015 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2015-')]
rankingyear2014 = filtered_rows[filtered_rows['STANDINGSDATE'].str.startswith('2014-')]

# Assuming you have already defined rankingyear2022, rankingyear2021, ..., rankingyear2014

# Concatenate the DataFrames along the rows
dff = pd.concat([rankingyear2022, rankingyear2021, rankingyear2020, rankingyear2019, rankingyear2018, rankingyear2017, rankingyear2016, rankingyear2015, rankingyear2014])

# Display the concatenated DataFrame
print(dff)


           TEAM_ID STANDINGSDATE         TEAM CONFERENCE
0       1610612743    2022-12-22       Denver       West
1       1610612763    2022-12-22      Memphis       West
2       1610612740    2022-12-22  New Orleans       West
3       1610612756    2022-12-22      Phoenix       West
4       1610612746    2022-12-22  LA Clippers       West
...            ...           ...          ...        ...
120052  1610612743    2014-12-22       Denver       West
120053  1610612758    2014-12-22   Sacramento       West
120054  1610612762    2014-12-22         Utah       West
120055  1610612747    2014-12-22  L.A. Lakers       West
120056  1610612750    2014-12-22    Minnesota       West

[135 rows x 4 columns]


In [25]:

dff.to_csv("CLASSEMENT")


In [34]:
import pandas as pd
import random  # Import the random module

# Read the CSV file into a DataFrame
df = pd.read_csv("CLASSEMENTT.csv")

# Define initial Elo ratings for each team
initial_elo = 1500  # Initial Elo rating
elo_ratings = {team: initial_elo for team in df['TEAM'].unique()}

# Define Elo rating update function based on rank group difference
def update_elo(rank_diff):
    K = 100  # Increased Elo rating update factor
    expected_win = 1 / (1 + 10 ** (rank_diff / 400))
    # Introduce randomness to Elo updates by adding a random value between -50 and 50

    return K * expected_win 

# Iterate over rows to update Elo ratings
for index, row in df.iterrows():
    team = row['TEAM']
    rank = (index % 15) + 1  # Calculate the rank dynamically based on the row index
    rank_group_diff = rank - 1  # Calculate the rank group difference
    elo_diff = update_elo(rank_group_diff)
    elo_ratings[team] += elo_diff

# Create a DataFrame for Elo ratings
elo_df = pd.DataFrame(elo_ratings.items(), columns=['TEAM', 'ELO'])
elo_df.drop(elo_df.tail(1).index, inplace=True)
sorted_df = elo_df.sort_values(by='ELO', ascending=False)
# Display the DataFrame
print(sorted_df)


             TEAM          ELO
10   Golden State  1945.971967
0          Denver  1943.812880
14        Houston  1942.375105
1         Memphis  1942.230641
11  Oklahoma City  1941.223414
13    San Antonio  1940.936297
8          Dallas  1940.504572
7        Portland  1940.503936
12    L.A. Lakers  1940.073609
6            Utah  1939.929023
3         Phoenix  1938.635064
2     New Orleans  1937.771778
5      Sacramento  1937.770997
9       Minnesota  1937.340154
4     LA Clippers  1895.683051
